### Uruchomienie testów:
Podstawowe komórki bezpośrednio wymagane do działania - podstawowe importy, basicstarter, cookiepopup, steamstart. Poza tym wszystkie funkcje są już niezależne, i mogą być uruchamiane w dowolnej kolejności, zgodnie z zasadą - najpierw inicjalizacja funkcji, następnie jej uruchomienie.

In [158]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.webdriver.support.ui import Select

### Przypadek testowy 1 (+ funkcja pomocnicza) - Sprawdzenie czy Steam zwraca poprawny tytuł przy uruchomieniu
Asercje: 1(1)

In [2]:
def basicstarter():
    options=webdriver.FirefoxOptions()
    driver=webdriver.Firefox(options=options)
    return driver

In [3]:
def closepopup(driver):
    cookie_popup = WebDriverWait(driver,15).until(expected_conditions.visibility_of_element_located((By.CSS_SELECTOR, '#cookiePrefPopup')))
    cookie = WebDriverWait(driver,10).until(expected_conditions.presence_of_element_located((By.CSS_SELECTOR, '#rejectAllButton > span:nth-child(1)')))
    cookie.click()

In [4]:
def steamstart(driver):
    driver.get("https://store.steampowered.com")
    assert "Welcome to Steam"==driver.title, "Tytuł nie pasuje"
    closepopup(driver)

In [29]:
def basickiller(driver):
    driver.quit()

In [32]:
def steamget():
    driver=basicstarter()
    steamstart(driver)
    basickiller(driver)
    print("Test zakończony powodzeniem")

In [33]:
steamget()

Test zakończony powodzeniem


### Przypadek Testowy 2 - sprawdzenie czy wyszukiwanie frazy Ghost Recon zwróci istotne wyniki
Asercje: 1(2)

In [34]:
def testGhostReconSearch():
    driver=basicstarter()
    steamstart(driver)
    search=driver.find_element(By.XPATH,'//input[@id="store_nav_search_term"]')
    search.send_keys("Ghost Recon")
    search.send_keys(Keys.RETURN)
    wait = WebDriverWait(driver, 10)
    first = wait.until(expected_conditions.presence_of_element_located((By.CSS_SELECTOR, 'a.search_result_row')))
    assert "GHOST RECON" in first.text.upper(), "Odnaleziony element nie jest istotny względem wyszukiwanej frazy"
    basickiller(driver)
    print("Test zakończony sukcesem")

In [35]:
testGhostReconSearch()

Test zakończony sukcesem


### Przypadek testowy 3 - sprawdzenie czy produkt na frontowym panelu może być istotny i czy pojawił się tam w związku z istotną obniżką ceny
Asercje: 6(8)

In [36]:
def checkfirstgame():
    driver=basicstarter()
    steamstart(driver)
    wait = WebDriverWait(driver, 10)
    first = wait.until(expected_conditions.presence_of_element_located((By.CSS_SELECTOR, 'a.tab_item')))
    lines=first.text.split("\n")
    assert "," in lines[-1], "Gra zawiera mniej niż 2 tagi - raczej nie będzie zbyt zaawansowana"
    assert "-" in lines[0], "Gra nie jest na zniżce"
    originalprice=lines[1]
    newprice=lines[2]
    originalprice=float(originalprice[:-2].replace(",", "."))
    newprice=float(newprice[:-2].replace(",","."))
    print("Gra -", lines[3])
    print("Zniżka -", lines[0])
    print("Cena oryginalna -", originalprice)
    print("Nowa cena -", newprice)
    print("-----------")
    if originalprice>=100:
        assert originalprice-newprice>=20, "Zniżka przy oryginalnej cenie powyżej 100 zł jest mniejsza niż 20 zł"
    elif originalprice>=50:
        assert originalprice-newprice>=10, "Zniżka przy oryginalnej cenie powyżej 50 zł jest mniejsza niż 10 zł"
    elif originalprice>=10:
        assert originalprice-newprice>=2, "Zniżka przy oryginalnej cenie powyżej 10 zł jest mniejsza niż 2 zł"
    else:
        assert originalprice-newprice>=1, "Zniżka jest mniejsza niż 1 zł"
    basickiller(driver)
    print("Test zakończony sukcesem")

In [37]:
checkfirstgame()

Gra - Sker Ritual
Zniżka - -20%
Cena oryginalna - 114.99
Nowa cena - 91.99
-----------
Test zakończony sukcesem


### Przypadek testowy 4 - Sprawdzenie czy przechodzenie między kategoriami ofert specjalnych działa poprawnie
Asercje: 4(12)

In [38]:
def checkspecials():
    driver=basicstarter()
    steamstart(driver)
    specialoffers = WebDriverWait(driver, 10).until(expected_conditions.element_to_be_clickable((By.LINK_TEXT, 'Specials')))
    specialoffers.click()
    assert "specials" in driver.current_url, "Całkowicie niewłaściwe przekierowanie"
    assert "tab=TopSellers" in driver.current_url, "Podstrona TopSellers nie została otwarta"
    casual = WebDriverWait(driver, 10).until(expected_conditions.element_to_be_clickable((By.CSS_SELECTOR, '#Tab_11')))
    casual.click()
    assert "tab=11" in driver.current_url, "Nie wykonano przejścia na kategorię Casual"
    sports = WebDriverWait(driver, 10).until(expected_conditions.element_to_be_clickable((By.CSS_SELECTOR, '#Tab_10')))
    sports.click()
    assert "tab=10" in driver.current_url, "Nie wykonano przejścia na kategorię Sports & Racing"
    basickiller(driver)
    print("Test zakończony sukcesem")

In [39]:
checkspecials()

Test zakończony sukcesem


### Przypadek testowy 5 - Sprawdzenie czy któreś z pierwszych dziesięciu najpopularniejszych gier na promocji pochodzi z lat 2023-2024
Asercje: 3(15)

In [40]:
def regularoffers():
    driver=basicstarter()
    steamstart(driver)
    specialoffers = WebDriverWait(driver, 10).until(expected_conditions.element_to_be_clickable((By.CSS_SELECTOR, 'a.big_button:nth-child(2)')))
    specialoffers.click()
    assert "https://store.steampowered.com/search/?specials=1"==driver.current_url, "Niewłaściwa strona Specials"
    games = WebDriverWait(driver,10).until(expected_conditions.presence_of_all_elements_located((By.CSS_SELECTOR, '#search_resultsRows')))
    games=games[0].text.split("\n")
    discounted=[]
    x=0
    assert len(games)>40, "Zbyt mało elementów na wyprzedaży..."
    for i in range (10):
        g1=games[x]
        x=x+1
        g2=games[x]
        x=x+1
        g3=games[x]
        x=x+1
        game=[]
        if g3[:-3].replace(",", "").isnumeric()==False:
            g4=games[x]
            x=x+1
            g5=games[x]
            x=x+1
            game=[g1, g2, g3, g4, g5]
        else:
            game=[g1, g2, g3]
        discounted.append(game)
    years=[]
    for i in range (10):
        year=discounted[i][1][-4:]
        years.append(year)
    assert (("2023" in years) or ("2024" in years)), "Brak najnowszych gier w najpopularniejszych produktach wyprzedażowych"
    basickiller(driver)
    print("Test zakończony sukcesem")

In [41]:
regularoffers()

Test zakończony sukcesem


### Przypadek testowy 6 - Sprawdzenie czy przejście do Steam Community działa poprawnie
Asercje: 2(17)

In [42]:
def getCommunity():
    driver=basicstarter()
    steamstart(driver)
    community = WebDriverWait(driver, 10).until(expected_conditions.element_to_be_clickable((By.CSS_SELECTOR, 'a.menuitem:nth-child(3)')))
    community.click()
    assert driver.current_url=="https://steamcommunity.com/", "Niewłaściwe przekierowanie"
    assert driver.title=="Steam Community", "Niewłaściwy tytuł strony"
    basickiller(driver)
    print("Test zakończony sukcesem")

In [43]:
getCommunity()

Test zakończony sukcesem


### Przypadek testowy 7 - Sprawdzenie czy przejście do About działa poprawnie
Asercje: 2(19)

In [44]:
def getAbout():
    driver=basicstarter()
    steamstart(driver)
    about = WebDriverWait(driver, 10).until(expected_conditions.element_to_be_clickable((By.CSS_SELECTOR, 'a.menuitem:nth-child(5)')))
    about.click()
    assert "https://store.steampowered.com/about/"==driver.current_url, "Niewłaściwe przekierowanie"
    assert driver.title=="Steam, The Ultimate Online Game Platform", "Niewłaściwy tytuł strony"
    basickiller(driver)
    print("Test zakończony sukcesem")

In [45]:
getAbout()

Test zakończony sukcesem


### Przypadek testowy 8 - Próba zmiany języka na polski
Asercje: 2(21)

In [46]:
def changeLanguage():
    driver=basicstarter()
    steamstart(driver)
    lang = WebDriverWait(driver, 10).until(expected_conditions.element_to_be_clickable((By.CSS_SELECTOR, '#language_pulldown')))
    lang.click()
    pl = WebDriverWait(driver, 10).until(expected_conditions.element_to_be_clickable((By.CSS_SELECTOR, 'a.tight:nth-child(19)')))
    pl.click()
    time.sleep(4)
    assert driver.current_url=="https://store.steampowered.com/", "Przekierowano na url inny niż strona sklepu"
    assert driver.title=="Witamy na Steam", "Tytuł nie odpowiada polskiemu tytułowi strony"
    basickiller(driver)
    print("Test zakończony sukcesem")

In [47]:
changeLanguage()

Test zakończony sukcesem


### Przypadek testowy 9 - Sprawdzenie czy kliknięcie na ikonę Steam spowoduje powrót na stronę główną
Asercje: 2(23)

In [48]:
def logopress():
    driver=basicstarter()
    steamstart(driver)
    logo = WebDriverWait(driver, 10).until(expected_conditions.element_to_be_clickable((By.CSS_SELECTOR, '#logo_holder > a:nth-child(1) > img:nth-child(1)')))
    logo.click()
    assert driver.current_url=="https://store.steampowered.com/", "Przekierowano na inną stronę"
    assert "Welcome to Steam"==driver.title, "Tytuł nie pasuje"
    basickiller(driver)
    print("Test zakończony sukcesem")

In [49]:
logopress()

Test zakończony sukcesem


### Przypadek testowy 10 - Sprawdzenie czy kliknięcie na ikonę Steam spowoduje powrót na stronę główną z innego obszaru(swoiste rozszerzenie testu 9)
Asercje: 4(29)

In [50]:
def logopress2():
    driver=basicstarter()
    steamstart(driver)
    points = WebDriverWait(driver, 10).until(expected_conditions.element_to_be_clickable((By.CSS_SELECTOR, 'a.tab:nth-child(8) > span:nth-child(1)')))
    points.click()
    assert driver.current_url=="https://store.steampowered.com/points/", "Nastąpiło przejście na niewłaściwy adres url"
    assert driver.title=="Steam Points Shop", "Strona ma niepoprawny tytuł"
    time.sleep(2)
    assert driver.current_url=="https://store.steampowered.com/points/shop", "Nie nastąpiła aktualizacja adresu url"
    logo = WebDriverWait(driver, 10).until(expected_conditions.element_to_be_clickable((By.CSS_SELECTOR, '#logo_holder > a:nth-child(1) > img:nth-child(1)')))
    logo.click()
    assert driver.current_url=="https://store.steampowered.com/", "Przekierowano na inną stronę niż główną"
    assert "Welcome to Steam"==driver.title, "Tytuł powrotnu nie pasuje"
    basickiller(driver)
    print("Test zakończony sukcesem")

In [51]:
logopress2()

Test zakończony sukcesem


### Przypadek testowy 11 - Wyszukanie gry i sprawdzenie jej ocen
Asercje: 2(31)

In [152]:
def lookforgame():
    driver=basicstarter()
    steamstart(driver)
    search=driver.find_element(By.XPATH,'//input[@id="store_nav_search_term"]')
    search.send_keys("Alone in the Dark 1")
    aitd= WebDriverWait(driver, 10).until(expected_conditions.element_to_be_clickable((By.CSS_SELECTOR, '.includes_games')))
    aitd.click()
    assert "Alone in the Dark 1 on Steam" in driver.title, "Niewłaściwa gra została wyświetlona"
    reviews= WebDriverWait(driver, 10).until(expected_conditions.element_to_be_clickable((By.CSS_SELECTOR, '#userReviews > div:nth-child(2) > div:nth-child(2) > span:nth-child(1)')))
    reviews.click()
    assert "#app_reviews_hash" in driver.current_url, "Nie udało się przejść automatycznie do ocen użytkowników"
    basickiller(driver)
    print("Test zakończony sukcesem")

In [153]:
lookforgame()

Test zakończony sukcesem


### Przypadek testowy 12 - Wyszukanie gry, otworzenie jej strony sklepu i przekierowanie do listy osiągnięć
Asercje: 3(34)

In [154]:
def achievements():
    driver=basicstarter()
    steamstart(driver)
    search=driver.find_element(By.XPATH,'//input[@id="store_nav_search_term"]')
    search.send_keys("new star soccer")
    search.send_keys(Keys.RETURN)
    first = WebDriverWait(driver, 10).until(expected_conditions.presence_of_element_located((By.CSS_SELECTOR, 'a.search_result_row')))
    assert "NEW STAR SOCCER 5" in first.text.upper(), "Nie odnaleziono szukanej gry"
    first.click()
    assert "New Star Soccer 5 on Steam" in driver.title, "Otworzono niewłaściwą grę"
    nss= WebDriverWait(driver, 10).until(expected_conditions.element_to_be_clickable((By.CSS_SELECTOR, 'a.communitylink_achievement')))
    nss.click()
    assert driver.title=="Steam Community :: New Star Soccer 5 :: Achievements", "Nie otworzono listy osiągnięć"
    basickiller(driver)
    print("Test zakończony sukcesem")

In [155]:
achievements()

Test zakończony sukcesem


### Przypadek testowy 13 - Wejście w Steam Refunds i uzyskanie dostępu do treści polityki zwrotów
Asercje: 3(37)

In [117]:
def refunds():
    deriver=basicstarter()
    steamstart(deriver)
    refunds= WebDriverWait(deriver, 10).until(expected_conditions.element_to_be_clickable((By.CSS_SELECTOR, '#footer_text > div:nth-child(2) > a:nth-child(4)')))
    refunds.click()
    deriver.switch_to.window(deriver.window_handles[1])
    time.sleep(2)
    assert deriver.title=="Steam Refunds", "Nie otworzono polityki zwrotów"
    content=deriver.page_source
    assert "refund period" in content.lower(), "Brak informacji o długości okresu zwrotów"
    assert "last updated" in content.lower(), "Brak informacji o dacie ostatniej aktualizacji polityki zwrotów"
    basickiller(deriver)
    print("Test zakończony sukcesem")

In [119]:
refunds()

Test zakończony sukcesem


### Przypadek testowy 14 - Sprawdzenie czy wejście w "Gry Polecane przez Znajomych" jako gość spowoduje przekierowanie na stronę "Zaloguj Się"
Asercje: 2(39)

In [120]:
def recommendedtoguest():
    driver=basicstarter()
    steamstart(driver)
    recommended= WebDriverWait(driver, 10).until(expected_conditions.element_to_be_clickable((By.CSS_SELECTOR, 'div.gutter_items:nth-child(6) > a:nth-child(1)')))
    recommended.click()
    assert "Sign In" == driver.title, "Tytuł strony nie odpowiada oczekiwanemu tytułowi"
    assert "https://store.steampowered.com/login" in driver.current_url, "Nie doszło do przekierowania na właściwą stronę logowania"
    basickiller(driver)
    print("Test zakończony sukcesem")

In [121]:
recommendedtoguest()

Test zakończony sukcesem


### Przypadek testowy 15 - Sprawdzenie czy przejście na stronę Steam Support i wyszukiwanie pomocy po wyrazie działa poprawnie
Asercje: 3(42)

In [146]:
def steamsupport():
    driver=basicstarter()
    steamstart(driver)
    support= WebDriverWait(driver, 10).until(expected_conditions.element_to_be_clickable((By.CSS_SELECTOR, '.supernav_container > a:nth-child(6)')))
    support.click()
    assert "Steam Support" == driver.title, "Tytuł strony jest niepoprawny"
    assert "https://help.steampowered.com/" in driver.current_url, "Nie doszło do przekierowania na właściwą stronę"
    search=WebDriverWait(driver, 10).until(expected_conditions.presence_of_element_located((By.CSS_SELECTOR, '#help_search_support_input')))
    search.send_keys("password")
    search.send_keys(Keys.RETURN)
    result=WebDriverWait(driver, 10).until(expected_conditions.presence_of_element_located((By.CSS_SELECTOR, 'a.help_search_result:nth-child(2)')))
    assert "password" in result.text and "change" in result.text, "Zmiana hasła nie jest pierwszym wynikiem wyszukiwania"
    basickiller(driver)
    print("Test zakończony sukcesem")

In [147]:
steamsupport()

Test zakończony sukcesem


### Przypadek testowy 16 - Próba przejścia na stronę gry z ograniczeniem wiekowym i otwarcie strony DLC do niej
Asercje: 2(44)

In [168]:
def goingpostal():
    driver=basicstarter()
    steamstart(driver)
    search=driver.find_element(By.XPATH,'//input[@id="store_nav_search_term"]')
    search.send_keys("postal iii")
    dude= WebDriverWait(driver, 10).until(expected_conditions.element_to_be_clickable((By.CSS_SELECTOR, 'a.match:nth-child(1)')))
    dude.click()
    assert "Postal III on Steam" in driver.title, "Niewłaściwa strona produktu"
    bypass = Select(driver.find_element(By.ID, 'ageYear'))
    bypass.select_by_value('1966')
    whatever = Select(driver.find_element(By.ID, "ageMonth"))
    whateva = Select(driver.find_element(By.ID, "ageDay"))
    whatever.select_by_value("June")
    whateva.select_by_value("6")
    driver.find_element(By.CSS_SELECTOR,'#view_product_page_btn > span:nth-child(1)').click()
    ost = WebDriverWait(driver, 10).until(expected_conditions.element_to_be_clickable((By.CSS_SELECTOR, '#dlc_row_2674620')))
    ost.click()
    assert "Postal 3 - Official Soundtrack on Steam"==driver.title, "Nie dotarto na stronę DLC"
    basickiller(driver)
    print("Test zakończony sukcesem")

In [169]:
goingpostal()

Test zakończony sukcesem


### Przypadek testowy 17 - Próba wejścia na rankingi graczy w grze owe rankingi udostępniającej
Asercje: 4(48)

In [181]:
def heybrother():
    driver=basicstarter()
    steamstart(driver)
    search=driver.find_element(By.XPATH,'//input[@id="store_nav_search_term"]')
    search.send_keys("AVICII")
    tim=WebDriverWait(driver, 10).until(expected_conditions.element_to_be_clickable((By.CSS_SELECTOR, 'a.match:nth-child(1)')))
    tim.click()
    assert "AVICII Invector on Steam" in driver.title, "Niewłaściwa strona gry"
    bergling=WebDriverWait(driver, 10).until(expected_conditions.element_to_be_clickable((By.CSS_SELECTOR, '.communitylink_achievement_images > a:nth-child(4)')))
    bergling.click()
    assert "Steam Community :: AVICII Invector :: Achievements" == driver.title, "Nie dotarto na stronę osiągnięć"
    true=WebDriverWait(driver, 10).until(expected_conditions.element_to_be_clickable((By.CSS_SELECTOR, '.tabOff > a:nth-child(1)')))
    true.click()
    assert "leaderboards" in driver.current_url, "Nie dotarto na stronę rankingów gry"
    gonnaloveya=Select(driver.find_element(By.CSS_SELECTOR, '#lbID'))
    gonnaloveya.select_by_value("3099556")
    stories=WebDriverWait(driver, 10).until(expected_conditions.element_to_be_clickable((By.CSS_SELECTOR, '#showButton')))
    stories.click()
    assert "Gonna Love Ya" in driver.title, "Nie dotarto do właściwego rankingu"
    basickiller(driver)
    print("Test zakończony sukcesem")

In [182]:
heybrother()

Test zakończony sukcesem


### Przypadek testowy 18 - Próba wejścia w dyskusje na temat konkretnej gry z jej strony sklepu
Asercje: 2(50)

In [192]:
def gamediscussions():
    driver=basicstarter()
    steamstart(driver)
    search=driver.find_element(By.XPATH,'//input[@id="store_nav_search_term"]')
    search.send_keys("ghost recon island")
    game=WebDriverWait(driver, 10).until(expected_conditions.element_to_be_clickable((By.CSS_SELECTOR, 'a.match:nth-child(1)')))
    game.click()
    assert "Tom Clancy's Ghost Recon® Island Thunder™ on Steam" in driver.title, "Niewłaściwa strona gry"
    discussions=WebDriverWait(driver, 10).until(expected_conditions.element_to_be_clickable((By.CSS_SELECTOR, 'a.linkbar:nth-child(4)')))
    discussions.click()
    driver.switch_to.window(driver.window_handles[1])
    time.sleep(2)
    assert "Tom Clancy's Ghost Recon: Island Thunder General Discussions :: Steam Community" == driver.title, "Nie dotarto do strony z dyskusjami dotyczącymi gry"
    basickiller(driver)
    print("Test zakończony sukcesem")

In [193]:
gamediscussions()

Test zakończony sukcesem


### Przypadek testowy 19 - Próba otworzenia strony sklepu z czterostronicowego, sześcioelementowego panelu ofert specjalnych
Asercje: 1(51)

In [216]:
def sixelements():
    driver=basicstarter()
    steamstart(driver)
    fourthpart=WebDriverWait(driver, 10).until(expected_conditions.element_to_be_clickable((By.CSS_SELECTOR, '#spotlight_carousel > div:nth-child(3) > div:nth-child(1)')))
    fourthpart.click()
    somegame=WebDriverWait(driver, 10).until(expected_conditions.element_to_be_clickable((By.CSS_SELECTOR, 'div.home_special_offers_group:nth-child(4) > div:nth-child(3) > div:nth-child(1) > a:nth-child(1) > div:nth-child(1) > img:nth-child(1)')))
    somegame.click()
    assert "on Steam" in driver.title, "Nie dotarto do strony gry"
    try:
        title = WebDriverWait(driver, 10).until(expected_conditions.presence_of_element_located((By.CSS_SELECTOR, "#appHubAppName")))
        print(title.text)
        print("------------")
    except:
        print("Najprawdopodobniej natrafiono na pakiet gier")
    basickiller(driver)
    print("Test zakończono sukcesem")

In [217]:
sixelements()

Red Dead Redemption 2
------------
Test zakończono sukcesem


### Przypadek testowy 20 - Próba otworzenia strony ustawień cookies i wyłączenie UTM
Asercje: 2(53)

In [229]:
def notracking():
    driver=basicstarter()
    steamstart(driver)
    cookiemonster=WebDriverWait(driver, 10).until(expected_conditions.element_to_be_clickable((By.CSS_SELECTOR, '#footer_text > div:nth-child(2) > a:nth-child(5)')))
    cookiemonster.click()
    driver.switch_to.window(driver.window_handles[1])
    time.sleep(2)
    templeofdoom=WebDriverWait(driver, 10).until(expected_conditions.presence_of_element_located((By.CSS_SELECTOR, "#main_content > div:nth-child(1) > div:nth-child(1)"))).text
    assert "Cookies" in templeofdoom, "Nie udało się dostać do ustawień cookies"
    utmbye=WebDriverWait(driver, 10).until(expected_conditions.element_to_be_clickable((By.CSS_SELECTOR, 'div._25H3qBQ1Lfsfq8nwc3M0Fk:nth-child(1) > div:nth-child(1) > div:nth-child(3) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1)')))
    utmbye.click()
    classbyebye=utmbye.get_attribute("class")
    assert classbyebye == "DialogToggleField_Option Off Active", "Nie udało się wyłączyć UTM"
    basickiller(driver)
    print("Test zakończono sukcesem")

In [230]:
notracking()

Test zakończono sukcesem
